# Prediction system

## This is a python notebook where you can upload your 2D turning data and get prediction results!

In [16]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define sit-to-stand features
features = [f'x{i}' for i in range(0, 17)] + [f'y{i}' for i in range(0, 17)] + ['number_of_turning_steps'] + ['turning_duration'] + ['turning_angle'] + ['type_of_turn']

# Define the LSTM classifier
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_patients, output_dim=2, num_layers=1):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.patient_embedding = nn.Embedding(num_patients, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, patient_ids):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        patient_intercepts = self.patient_embedding(patient_ids)
        out = out + patient_intercepts
        out = self.fc(out)
        return out

# Load the scaler fitted to the model
scaler_file_path = '/Users/suhrudp/Library/CloudStorage/OneDrive-Personal/Stats/REMAP Open Dataset PD/Analysis/Models/scaler_turn2d.pkl'
with open(scaler_file_path, 'rb') as file:
    scaler = pickle.load(file)

# Load the LSTM model weights
model_file_path = '/Users/suhrudp/Library/CloudStorage/OneDrive-Personal/Stats/REMAP Open Dataset PD/Analysis/Models/model_turn2d.pth'
model = LSTMClassifier(input_dim=38, num_patients=22, hidden_dim=59, num_layers=2) # model trained using data from 22 patients
model.load_state_dict(torch.load(model_file_path))
model.eval()

# Define data preprocessing as done while training the model
def preprocess_new_data(filepath, scaler, turning_features=None):
    new_data = pd.read_csv(filepath, header=None, skiprows=1)
    column_names = [f'x{i}' for i in range(0, 17)] + [f'y{i}' for i in range(0, 17)]
    new_data.columns = column_names
    new_data = new_data.iloc[:, 2:]
    expected_features = [f'x{i}' for i in range(0, 17)] + [f'y{i}' for i in range(0, 17)] + [
        'number_of_turning_steps', 'turning_duration', 'turning_angle', 'type_of_turn']
    if turning_features is None:
        turning_features = {
            'number_of_turning_steps': 0,
            'turning_duration': 0.0,
            'turning_angle': 0.0,
            'type_of_turn': 0
        }
    for feature, default_value in turning_features.items():
        if feature not in new_data.columns:
            new_data[feature] = default_value
    for feature in expected_features:
        if feature not in new_data.columns:
            new_data[feature] = 0
    new_data = new_data[expected_features]
    new_data[expected_features] = scaler.transform(new_data[expected_features])
    return new_data

# Define the model prediction function
def predict(model, new_data, patient_ids):
    with torch.no_grad():
        new_data = torch.tensor(new_data.to_numpy(dtype=np.float32))
        patient_ids = torch.tensor(patient_ids, dtype=torch.long)
        if new_data.ndim == 2:
            new_data = new_data.unsqueeze(0)
            patient_ids = patient_ids.unsqueeze(0)
        outputs = model(new_data, patient_ids)
        _, predicted = torch.max(outputs.data, 1)
        predictions = ["PD" if label == 1 else "Control" for label in predicted.tolist()]
        return predictions

# Prediction using the model
filepath = '/Users/suhrudp/Library/CloudStorage/OneDrive-Personal/Stats/REMAP Open Dataset PD/21h9f9e30v9cl2fapjggz4q1x7/Turning/Data/turning_2D3D_skeletons_coarsened/Turning_coarsen_CSV/Pt204_C_n_387/input_2D/keypoints.csv'
turning_features = {
    'number_of_turning_steps': 5,
    'turning_duration': 1.5,
    'turning_angle': 4,
    'type_of_turn': 1
}
preprocessed_data = preprocess_new_data(filepath, scaler, turning_features=turning_features)
patient_ids = np.array([0])
prediction = predict(model, preprocessed_data, patient_ids)
print(f'The predictions for the new data are: {prediction}')

The predictions for the new data are: ['Control']
